In [3]:
# Считываем данные с файлов
import pandas as pd
users = pd.read_table('users.dat', sep='::', names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
movies = pd.read_table('movies.dat', sep='::', names=['MovieID', 'Title', 'Genres'])
ratings =pd.read_table('ratings.dat', sep='::', names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

<ipython-input-3-be0e72fe7146>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_table('users.dat', sep='::', names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
<ipython-input-3-be0e72fe7146>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('movies.dat', sep='::', names=['MovieID', 'Title', 'Genres'])
<ipython-input-3-be0e72fe7146>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying eng

In [4]:
# Объединяем данные в одну таблицу с помощью merge
first = pd.merge(ratings, users)
table = pd.merge(first, movies)
print(table)

         UserID  MovieID  Rating  Timestamp Gender  Age  Occupation Zip-code  \
0             1     1193       5  978300760      F    1          10    48067   
1             2     1193       5  978298413      M   56          16    70072   
2            12     1193       4  978220179      M   25          12    32793   
3            15     1193       4  978199279      M   25           7    22903   
4            17     1193       5  978158471      M   50           1    95350   
...         ...      ...     ...        ...    ...  ...         ...      ...   
1000204    5949     2198       5  958846401      M   18          17    47901   
1000205    5675     2703       3  976029116      M   35          14    30030   
1000206    5780     2845       1  958153068      M   18          17    92886   
1000207    5851     3607       5  957756608      F   18          20    55410   
1000208    5938     2909       4  957273353      M   25           1    35401   

                                       

In [5]:
# Первая таблица - средний рейтинг всех фильмов при групперовке по полу
# Вторая таблица - средний рейтинг всех фильмов при групперовке по возрасту
Gender_Rating = pd.pivot_table(table,
               index=["Gender", 'Title'],
              values=['Rating'])
print(Gender_Rating, '\n')
Age_Rating = pd.pivot_table(table,
               index=["Age", 'Title'],
              values=['Rating'])
print(Age_Rating)

                                                     Rating
Gender Title                                               
F      $1,000,000 Duck (1971)                      3.375000
       'Night Mother (1986)                        3.388889
       'Til There Was You (1997)                   2.675676
       'burbs, The (1989)                          2.793478
       ...And Justice for All (1979)               3.828571
...                                                     ...
M      Zed & Two Noughts, A (1985)                 3.380952
       Zero Effect (1998)                          3.723140
       Zero Kelvin (Kj�rlighetens kj�tere) (1995)  3.500000
       Zeus and Roxanne (1997)                     2.357143
       eXistenZ (1999)                             3.289086

[7152 rows x 1 columns] 

                                             Rating
Age Title                                          
1   'Night Mother (1986)                   2.000000
    'Til There Was You (1997)        

In [6]:
# В первой таблице мы отсортировали средние рейтинги в порядке убывания популярности у женской аудитории.
Gender_Rating_FemSort = Gender_Rating['F':'F'].sort_values(by='Rating', ascending=False)
print(Gender_Rating_FemSort)
# С сортировкой при группировке по возрасту возникли проблемы (было невозможно выбрать всех 56-летних людей, если возраст был индексом) поэтому я использовал костыль:
Age_Rating_T = Age_Rating.T
Age_Rating = Age_Rating_T[56].T
Age_Rating_OldSort = Age_Rating.sort_values(by='Rating', ascending=False)
print(Age_Rating_OldSort)

                                                           Rating
Gender Title                                                     
F      Clean Slate (Coup de Torchon) (1981)                   5.0
       Ballad of Narayama, The (Narayama Bushiko) (1958)      5.0
       Raw Deal (1948)                                        5.0
       Bittersweet Motel (2000)                               5.0
       Skipped Parts (2000)                                   5.0
...                                                           ...
       My Life and Times With Antonin Artaud (En compa...     1.0
       Sorority House Massacre II (1990)                      1.0
       Big Bully (1996)                                       1.0
       Dangerous Ground (1997)                                1.0
       Bloodsport 2 (1995)                                    1.0

[3481 rows x 1 columns]
                                             Rating
Title                                              
Jude (1996)  

In [7]:
func_count = {
    'Rating':'count'
}
a_lot_of_ratings = table.groupby(['Title']).agg(func_count)
filtered_a_lot_of_ratings = a_lot_of_ratings.query('Rating>=250')
print(filtered_a_lot_of_ratings)

                                   Rating
Title                                    
'burbs, The (1989)                    303
10 Things I Hate About You (1999)     700
101 Dalmatians (1961)                 565
101 Dalmatians (1996)                 364
12 Angry Men (1957)                   616
...                                   ...
Young Guns (1988)                     562
Young Guns II (1990)                  369
Young Sherlock Holmes (1985)          379
Zero Effect (1998)                    301
eXistenZ (1999)                       410

[1216 rows x 1 columns]


In [8]:
mid_func = {
    'Rating':'mean'
}
cry_func = {
    'Rating':'std'
}
MiddleValue = table.groupby(['Title', 'Gender']).agg(mid_func)
print(MiddleValue)
what_is_that = {
    'Rating':'min'
}
what_is_that2 = {
    'Rating':'max'
}
minrating = MiddleValue.groupby(['Title']).agg(what_is_that)
maxrating = MiddleValue.groupby(['Title']).agg(what_is_that2)
wat = maxrating - minrating
wat_sorted = wat.sort_values(by='Rating', ascending=False)
Gender_Rating_MaleSort = Gender_Rating['M':'M'].sort_values(by='Rating', ascending=False)
Gender_Rating_MaleSort.reset_index(level='Gender', drop=True, inplace=True)
i_am_crying = Gender_Rating_FemSort - Gender_Rating_MaleSort
i_am_crying = i_am_crying.sort_values(by='Rating', ascending=False)
print('Популярные у женщин, но ненавистные мужчинами')
print(i_am_crying.head(15))
i_am_crying = i_am_crying.sort_values(by='Rating', ascending=True)
print('Популярные у мужчин, но ненавистные женщинами')
print(i_am_crying.head(15))

                                                     Rating
Title                                      Gender          
$1,000,000 Duck (1971)                     F       3.375000
                                           M       2.761905
'Night Mother (1986)                       F       3.388889
                                           M       3.352941
'Til There Was You (1997)                  F       2.675676
...                                                     ...
Zero Kelvin (Kj�rlighetens kj�tere) (1995) M       3.500000
Zeus and Roxanne (1997)                    F       2.777778
                                           M       2.357143
eXistenZ (1999)                            F       3.098592
                                           M       3.289086

[7152 rows x 1 columns]
Популярные у женщин, но ненавистные мужчинами
                                                             Rating
Gender Title                                                       
F      Spider